In [7]:
from getpass import getpass
from coreapi.auth import BasicAuthentication
from quantuminspire.api import QuantumInspireAPI
from quantuminspire.credentials import load_account, get_token_authentication, get_basic_authentication
from quantuminspire.api import QuantumInspireAPI
from qiskit import IBMQ, QuantumRegister, ClassicalRegister, QuantumCircuit, assemble, transpile
from qiskit.providers.ibmq import least_busy
from qiskit.circuit import Parameter 



import numpy as np
import pandas as pd
import plotly.express as px

In [8]:
theta_param, phi_param = Parameter('theta_param'), Parameter('phi_param') 

qreg_q = QuantumRegister(2, 'q')
creg_c = ClassicalRegister(2, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)

circuit.ry(theta_param, qreg_q[0])
circuit.rz(phi_param, qreg_q[0])

circuit.rx(-np.pi/2, qreg_q[0])
circuit.sdg(qreg_q[1])
circuit.h(qreg_q[1])
circuit.tdg(qreg_q[1])
circuit.cx(qreg_q[0], qreg_q[1])
circuit.t(qreg_q[1])
circuit.h(qreg_q[1])
circuit.s(qreg_q[1])
circuit.cx(qreg_q[1], qreg_q[0])
circuit.rx(np.pi/2, qreg_q[0])
circuit.rx(np.pi/2, qreg_q[1])

circuit.rz(-phi_param, qreg_q[0])
circuit.ry(-theta_param, qreg_q[1])
circuit.measure(qreg_q[0], creg_c[0])
circuit.measure(qreg_q[1], creg_c[1])

In [9]:
N_shots=8000
provider = IBMQ.load_account()

/home/neuromancer/anaconda3/lib/python3.7/site-packages/qiskit/providers/ibmq/ibmqfactory.py:192: UserWarning:

Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.

ibmqfactory.load_account:WARNING:2021-01-04 21:55:09,495: Credentials are already in use. The existing account in the session will be replaced.


In [10]:
num_pts = 100

coords= [(0,0),(np.pi,0),(np.pi/2,0),(np.pi/2,np.pi)]

x=[np.cos(point[1])*np.sin(point[0]) for point in coords]
y=[np.sin(point[1])*np.sin(point[0]) for point in coords]
z=[np.cos(point[0]) for point in coords]

#x, y, z = np.cos(phi) * np.sin(theta), np.sin(phi) * np.sin(theta), np.cos(theta);

# Plot the surface.
df = px.data.iris()
fig = px.scatter_3d(df, x, y, z, template="plotly_white")
fig.update_layout(
    scene = dict(
        xaxis = dict(range=[-1,1],),
        yaxis = dict(range=[-1,1],),
        zaxis = dict(range=[-1,1],)
    )
)
fig.update_layout(scene_aspectmode='cube')
fig.show()
target_points=np.array(coords)


In [15]:
def run():
    runningjobs=[]
    job_ids=[]
    index=0

    while index*75 < num_pts:
        if((index+1)*75<num_pts):
            circuits = [circuit.bind_parameters(
            {theta_param: points[0], phi_param : points[1]}) 
             for points in coords[index*75:(index+1)*75]]
            qobj = assemble(transpile(circuits, backend=backend), backend=backend, shots=N_shots)
            runningjobs.append(backend.run(qobj))
            job_ids.append(runningjobs[-1].job_id())
            with open(id_file, 'a') as file:
                file.write(runningjobs[-1].job_id()+"\n")
            if(backend.job_limit().active_jobs == backend.job_limit().maximum_jobs):
                print("Maximum number of jobs reached. Waiting...")
                try:
                    job_result = runningjobs[0].result()  # It will block until the job finishes.
                    print("The job finished with result {}".format(job_result))
                    runningjobs.pop(0)
                except JobError as ex:
                    print("Something wrong happened!: {}".format(ex))
        else:
            circuits = [circuit.bind_parameters(
            {theta_param: points[0], phi_param: points[1]}) 
             for points in coords[index*75:]]
            qobj = assemble(transpile(circuits, backend=backend), backend=backend, shots=N_shots)
            runningjobs.append(backend.run(qobj))
            job_ids.append(runningjobs[-1].job_id())
            with open(id_file, 'a') as file:
                file.write(runningjobs[-1].job_id()+"\n")
            if(backend.job_limit().active_jobs == backend.job_limit().maximum_jobs):
                print("Maximum number of jobs reached. Waiting...")
                try:
                    job_result = runningjobs[0].result()  # It will block until the job finishes.
                    print("The job finished with result {}".format(job_result))
                    runningjobs.pop(0)
                except JobError as ex:
                    print("Something wrong happened!: {}".format(ex))
        with open(id_file, 'a') as file:
            file.write(runningjobs[-1].job_id()+"\n")
        index=index+1

In [19]:
def post_process():
    read_job_ids = open(id_file).read().splitlines()
    read_job_ids = list(set(read_job_ids))
    counts_array=[]
    results_probabilities=[]
    retrieved_jobs=[backend.retrieve_job(jobid) for jobid in read_job_ids]
    for job in retrieved_jobs:
        counts_array=counts_array+job.result().get_counts()
    for job in counts_array:
        #test both the copies
        prob0_qubit_0=job["00"]/N_shots+job["01"]/N_shots
        prob1_qubit_0=job["10"]/N_shots+job["11"]/N_shots
        prob0_qubit_1=job["00"]/N_shots+job["10"]/N_shots
        prob1_qubit_1=job["01"]/N_shots+job["11"]/N_shots
        results_probabilities.append([prob0_qubit_0,prob1_qubit_0,prob0_qubit_1,prob1_qubit_1])

    results_probabilities=np.array(results_probabilities)

    data_sheet=pd.DataFrame(data=np.hstack((target_points,results_probabilities)),columns=["θ","ϕ","prob_0_qubit_0","prob_1_qubit_0","prob_0_qubit_1","prob_1_qubit_1"])

    data_sheet.to_excel(data_sheet_file)

    #FIDELITY OF FIRST COPY
    # Make data.
    thetas=data_sheet["θ"]
    phis=data_sheet["ϕ"]
    x, y, z = np.cos(phis) * np.sin(thetas), np.sin(phis) * np.sin(thetas), np.cos(thetas);

    # Plot the surface. F_measured/F_theory
    df = px.data.iris()
    fig = px.scatter_3d(df, x, y, z, color=data_sheet.prob_0_qubit_0, template="plotly_white")
    fig.update_layout(
        scene = dict(
            xaxis = dict(range=[-1,1],),
            yaxis = dict(range=[-1,1],),
            zaxis = dict(range=[-1,1],)
        )
    )
    fig.update_layout(scene_aspectmode='cube')
    fig.show()
    print("Number of points: ", len(x))
    print("Average fidelity: ", np.average(data_sheet.prob_0_qubit_0))
    print("Standard deviation: ", np.std(data_sheet.prob_0_qubit_0))

In [30]:
backend = provider.backends.ibmq_santiago
id_file = 'eco_qcm_ids_santiago_2.txt'
result_txt='eco_results_ibm_santiago_2.txt'
data_sheet_file="eco_data_sheet_ibm_santiago_2.xlsx"
try:
    run()
except:
    print("done")

done


/home/neuromancer/anaconda3/lib/python3.7/site-packages/qiskit/compiler/transpile.py:178: UserWarning:

Transpiling schedules is not supported yet.



In [29]:
post_process()

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 4 and the array at index 1 has size 20

In [ ]:
backend = provider.backends.ibmq_athens
id_file = 'eco_qcm_ids_athens_2.txt'
result_txt='eco_results_ibm_athens_2.txt'
data_sheet_file="eco_data_sheet_ibm_athens_2.xlsx"
try:
    run()
except:
    print("done")
    post_process()

In [ ]:
backend = provider.backends.ibmq_vigo
id_file = 'eco_qcm_ids_vigo_2.txt'
result_txt='eco_results_ibm_vigo_2.txt'
data_sheet_file="eco_data_sheet_ibm_vigo_2.xlsx"
try:
    run()
except:
    print("done")
    post_process()

In [ ]:
backend = provider.backends.ibmq_ourense
id_file = 'eco_qcm_ids_ourense_2.txt'
result_txt='eco_results_ibm_ourense_2.txt'
data_sheet_file="eco_data_sheet_ibm_ourense_2.xlsx"
try:
    run()
except:
    print("done")

In [22]:
backend = provider.backends.ibmq_valencia
id_file = 'eco_qcm_ids_valencia_2.txt'
result_txt='eco_results_ibm_valencia_2.txt'
data_sheet_file="eco_data_sheet_ibm_valencia_2.xlsx"
try:
    run()
except:
    print("done")

done


/home/neuromancer/anaconda3/lib/python3.7/site-packages/qiskit/compiler/transpile.py:178: UserWarning:

Transpiling schedules is not supported yet.



In [ ]:
backend = provider.backends.ibmqx2
id_file = 'eco_qcm_ids_yorktown.txt'
result_txt='eco_results_ibm_yorktown.txt'
data_sheet_file="eco_data_sheet_ibm_yorktown.xlsx"
try:
    run()
except:
    print("done")

In [ ]:
backend = provider.backends.ibmq_16_melbourne
id_file = 'eco_qcm_ids_melbourne.txt'
result_txt='eco_results_ibm_melbourne.txt'
data_sheet_file="eco_data_sheet_ibm_melbournen.xlsx"
try:
    run()
except:
    print("done")